In [1]:
import os
import copy
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import multivariate_normal

from microsim.population import (NHANESDirectSamplePopulation, 
                                 build_people_using_nhanes_for_sampling,
                                 microsimToNhanes, Population, get_nhanes_population)
from microsim.risk_model_repository import RiskModelRepository
from microsim.gender import NHANESGender
from microsim.smoking_status import SmokingStatus
from microsim.race_ethnicity import NHANESRaceEthnicity
from microsim.education import Education
from microsim.treatment import DefaultTreatmentsType,  TreatmentStrategiesType
from microsim.risk_factor import StaticRiskFactorsType, DynamicRiskFactorsType
from microsim.afib_model import AFibPrevalenceModel
from microsim.pvd_model import PVDPrevalenceModel
from microsim.outcome import Outcome, OutcomeType
from microsim.person import Person
from microsim.population_model_repository import PopulationModelRepository
from microsim.cohort_risk_model_repository import (CohortStaticRiskFactorModelRepository,
                                                   CohortDynamicRiskFactorModelRepository,
                                                   CohortDefaultTreatmentModelRepository)
from microsim.outcome_model_repository import OutcomeModelRepository

microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM"
os.chdir(microsimDir+"/CODE/microsim")

dataDir = microsimDir+"/NOTEBOOKS/DATA"

In [2]:
categoricalVars = ["gender", "smokingStatus", "raceEthnicity", "statin",'education',
                  'alcoholPerWeek','anyPhysicalActivity','antiHypertensiveCount']
continuousVars = ['age', 'hdl', 'bmi', 'totChol', 'trig', 'a1c', 'ldl', 'waist', 'creatinine', 'sbp', 'dbp']

In [3]:
nhanesDf = pd.read_stata(microsimDir + "/CODE/microsim/microsim/data/fullyImputedDataset.dta")
nhanesDf = nhanesDf.rename(columns={"level_0":"name"})

In [4]:
df = pd.read_csv(dataDir+"/nhanes-persons-from-Gaussians.csv")
df.head()

,name,gender,smokingStatus,raceEthnicity,statin,education,alcoholPerWeek,anyPhysicalActivity,antiHypertensiveCount,age,hdl,bmi,totChol,trig,a1c,ldl,waist,creatinine,sbp,dbp
0,41909,1,0,1,False,1,0,0,0.0,62.157082,45.814063,36.589655,226.430582,108.345160,6.263221,155.639129,123.571332,0.989469,122.375309,65.381554
1,42688,1,0,1,False,1,0,0,0.0,49.992311,44.946865,28.189541,153.619603,110.972330,5.859257,88.971266,102.506521,0.593414,125.949251,88.682632
2,43025,1,0,1,False,1,0,0,0.0,15.983673,39.375618,33.380314,234.115627,251.663072,5.772535,139.664001,109.788199,0.851678,111.377956,87.864916
3,43390,1,0,1,False,1,0,0,0.0,38.715531,39.940968,38.184832,185.337142,242.284426,5.553624,92.645844,127.502329,0.894123,123.191501,65.891548
4,44501,1,0,1,False,1,0,0,0.0,45.939015,40.457015,22.634844,160.320499,172.850585,5.664377,81.183563,85.895882,0.600087,145.303193,76.534771


In [5]:
df.shape

(5448, 20)

In [6]:
df = df.merge(nhanesDf[["name","WTINT2YR"]], on="name", how="inner").copy()
df.shape

(5448, 21)

In [7]:
def build_person(x, initializationModelRepository):
    """Takes all Person-instance-related data via x and initializationModelRepository and organizes it,
       passes the organized data to the Person class and returns a Person instance."""
    
    rng = np.random.default_rng()
        
    name = x.name
    
    personStaticRiskFactors = {
                        StaticRiskFactorsType.RACE_ETHNICITY.value: NHANESRaceEthnicity(int(x.raceEthnicity)),
                        StaticRiskFactorsType.EDUCATION.value: Education(int(x.education)),
                        StaticRiskFactorsType.GENDER.value: NHANESGender(int(x.gender)),
                        StaticRiskFactorsType.SMOKING_STATUS.value: SmokingStatus(int(x.smokingStatus))}
        
    #use this to get the bounds imposed on the risk factors in a bit
    rfRepository = RiskModelRepository()
        
    #TO DO: find a way to include everything here, including the rfs that need initialization
    #the PVD model would be easy to implement, eg with an estimate_next_risk_for_patient_characteristics function
    #but the AFIB model would be more difficult because it relies on statsmodel_logistic_risk file
    #for now include None, in order to create the risk factor lists correctly at the Person instance
    personDynamicRiskFactors = dict()
    for rfd in DynamicRiskFactorsType:
        #if rfd==DynamicRiskFactorsType.ALCOHOL_PER_WEEK:
        #    personDynamicRiskFactors[rfd.value] = AlcoholCategory.get_category_for_consumption(rfRepository.apply_bounds(rfd.value, x[microsimToNhanes[rfd.value]]))
        #else:
        if (rfd!=DynamicRiskFactorsType.PVD) & (rfd!=DynamicRiskFactorsType.AFIB):
            personDynamicRiskFactors[rfd.value] = rfRepository.apply_bounds(rfd.value, x[rfd.value])
    personDynamicRiskFactors[DynamicRiskFactorsType.AFIB.value] = None
    personDynamicRiskFactors[DynamicRiskFactorsType.PVD.value] = None

    #Q: do we need otherLipid treatment? I am not bringing it to the Person objects for now.
    #A: it is ok to leave it out as we do not have a model to update this. It is also very rarely taking place in the population anyway.
    #also: used to have round(x.statin) but NHANES includes statin=2...
    personDefaultTreatments = {
                        DefaultTreatmentsType.STATIN.value: bool(x.statin),
                        #DefaultTreatmentsType.OTHER_LIPID_LOWERING_MEDICATION_COUNT.value: x.otherLipidLowering,
                        DefaultTreatmentsType.ANTI_HYPERTENSIVE_COUNT.value: x.antiHypertensiveCount}

    personTreatmentStrategies = dict(zip([strategy.value for strategy in TreatmentStrategiesType],
                                         #[None for strategy in range(len(TreatmentStrategiesType))]))
                                         [{"status": None} for strategy in range(len(TreatmentStrategiesType))]))

    personOutcomes = dict(zip([outcome for outcome in OutcomeType],
                                  [list() for outcome in range(len(OutcomeType))]))
    #add pre-simulation stroke outcomes
    #selfReportStrokeAge=x.selfReportStrokeAge
    #Q: we should not add the stroke outcome in case of "else"?
    #if selfReportStrokeAge is not None and selfReportStrokeAge > 1:
    #        selfReportStrokeAge = selfReportStrokeAge if selfReportStrokeAge <= x.age else x.age
    #        personOutcomes[OutcomeType.STROKE].append((selfReportStrokeAge, StrokeOutcome(False, None, None, None, priorToSim=True)))
    #add pre-simulation mi outcomes
    #selfReportMIAge=rng.integers(18, x.age) if x.selfReportMIAge == 99999 else x.selfReportMIAge
    #if selfReportMIAge is not None and selfReportMIAge > 1:
    #        selfReportMIAge = selfReportMIAge if selfReportMIAge <= x.age else x.age
    #        personOutcomes[OutcomeType.MI].append((selfReportMIAge, Outcome(OutcomeType.MI, False, priorToSim=True)))

    person = Person(name,
                   personStaticRiskFactors,
                   personDynamicRiskFactors,
                   personDefaultTreatments,
                   personTreatmentStrategies,
                   personOutcomes)

    #TO DO: find a way to initialize these rfs above with everything else
    person._pvd = [initializationModelRepository[DynamicRiskFactorsType.PVD].estimate_next_risk(person)]
    person._afib = [initializationModelRepository[DynamicRiskFactorsType.AFIB].estimate_next_risk(person)]
    return person


In [8]:
initializationModelRepository = {DynamicRiskFactorsType.AFIB: AFibPrevalenceModel(), 
                                     DynamicRiskFactorsType.PVD: PVDPrevalenceModel()}

In [9]:
dfSample = df.sample(100000, weights=df.WTINT2YR, replace=True)

In [10]:
people = dfSample.apply(lambda x: build_person(x, initializationModelRepository), axis=1)

In [11]:
#sets the unique identifier for each Person instance
noneList = list(map(lambda person, i: setattr(person, "_index", i), people, range(len(people)))) 

In [12]:
popModelRepository = PopulationModelRepository(CohortDynamicRiskFactorModelRepository(),
                                                           CohortDefaultTreatmentModelRepository(),
                                                           OutcomeModelRepository(),
                                                           CohortStaticRiskFactorModelRepository()) 
pop = Population(people, popModelRepository)


In [13]:
pop._people.shape

(100000,)

In [14]:
#pop2 = get_nhanes_population(1999)
pop2 = NHANESDirectSamplePopulation(100000, 1999)
pop2.advance(1)
#pop2.print_baseline_summary()

In [15]:
pop.print_baseline_summary_comparison(pop2)

                                                     self  other
                                               age   40.9   43.8
                                               sbp  120.5  122.3
                                               dbp   71.0   72.2
                                               a1c    4.8    5.0
                                               hdl   51.4   50.8
                                               ldl  122.6  123.1
                                              trig  139.1  146.8
                                           totChol  200.9  201.6
                                               bmi   26.7   27.4
                               anyPhysicalActivity    0.7    0.7
                                              afib    0.0    0.0
                                             waist   93.2   94.6
                                    alcoholPerWeek    1.9    1.9
                                        creatinine    0.1    0.1
                         